In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn_crfsuite import CRF
# ref for first module https://scikit-learn.org/stable/modules/cross_validation.html
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn_crfsuite.metrics import flat_classification_report

In [2]:
def read_all_line( directory , file ):
    file = open( directory + "/" + file , "r" , encoding="utf-8" )
    contents = file.read()
    file.close()
    return contents

In [3]:
def word2features(sent, i):
    word = sent[i][0]
    dict_type = sent[i][2]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'dictionary' : dict_type
    }
    if i > 0:
        word1 = sent[i-1][0]
        dict_type1 = sent[i-1][2]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:dictionay' : dict_type1
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        dict_type1 = sent[i+1][2]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '-1:dictionay' : dict_type1
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label, dict_type in sent]

def sent2tokens(sent):
    return [token for token, label, dict_type in sent]

def sent2dict_type( sent ):
    return [dict_type for token, label, dict_type in sent]

In [4]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 0
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t, d) for w, t, d in zip( s["Word"].values.tolist(),
                                                            s["Tag"].values.tolist(),
                                                            s["Dictionary"].values.tolist() 
                                                          )]
        self.grouped = self.grouped_sentence()
        self.sentences = [s for s in self.grouped]
    
    def grouped_sentence( self ):
        save_sentence = 0
        major_index = 0
        temp_grouped = []
        for run in range( 0 , self.data.shape[0] ):
            temp_sentence = num_of_sentence( self.data["Sentence #"][ run ] )
            if save_sentence != temp_sentence :
                major_index = temp_sentence - 1
                save_sentence = temp_sentence
                temp_grouped.append( [] )
            temp_grouped[ major_index ].append( (self.data["Word"][run], 
                                                 self.data["Tag"][run],
                                                 self.data["Dictionary"][run] ) )
        return temp_grouped
    
    def get_next(self):
        try:
            s = self.grouped[ self.n_sent ]
            self.n_sent += 1
            return s  
        except:
            self.empty = True
            return None, None

In [5]:
def get_tag_group_sentence( data ):
    old_tag = []
    save_sentence = ""
    run_old = -1
    for run in range(  0 , data.shape[0] ):
        if( save_sentence != data["Sentence #"][ run ] ):
            old_tag.append( [] )
            run_old += 1
        else:
            None
        if data.Tag[run] in ( '' ):
            old_tag[ run_old ].append( 'O' )
        else:
            old_tag[ run_old ].append( data.Tag[ run ] )
    return old_tag

def merge_all_tag( data ):
    old_tag = []
    save_sentence = ""
    run_old = -1
    for run in range(  0 , data.shape[0] ):
        if( save_sentence != data["Sentence #"][ run ] ):
            old_tag.append( [] )
            run_old += 1
        else:
            None
        if data.Tag[run] in ( '' ):
            old_tag[ run_old ].append( 'O' )
            data.Tag[run] = 'O'
        elif data.Tag[run] in ('loc_cont','loc_end','loc_start'):
            old_tag[ run_old ].append( data.Tag[ run ] )
            data.Tag[run] = 'loc'
        elif data.Tag[run] in ('org_cont','org_end','org_start'):
            old_tag[ run_old ].append( data.Tag[ run ] )
            data.Tag[run] = 'org'
        elif data.Tag[run] in ('per_cont','per_end','per_start'):
            old_tag[ run_old ].append( data.Tag[ run ] )
            data.Tag[run] = 'per'
        else:
            old_tag[ run_old ].append( data.Tag[ run ] )
            None
    return old_tag

# number is current index
# target is -1 -2 +1 +2 what do you want to file
def helper_get_tag( data , number , target , limit ):
    if( number + target == limit ):
        return 'O'
    elif( data["Sentence #"][ number+target ] != data["Sentence #"][ number ] ):
        return 'O'
    else:
        return data.Tag[number + target ]
            
def split_all_tag( data ):
    limit_run = data.shape[0]
    for run in range( 0 , limit_run ):
        if data.Tag[ run ] == "loc" :
            if( helper_get_tag( data , run , -1, limit_run  ) in ( "loc_start" , "loc_cont" ) ):
                if( helper_get_tag( data , run , +1 , limit_run ) == "loc" ):
                    data.Tag[ run ] = "loc_cont"
                else:
                    data.Tag[ run ] = "loc_end"
            elif( helper_get_tag( data , run , +1 , limit_run ) == "loc" ):
                data.Tag[ run ] = "loc_start"
        elif data.Tag[ run ] == "org" :
            if( helper_get_tag( data , run , -1 , limit_run ) in ( "org_start" , "org_cont" ) ):
                if( helper_get_tag( data , run , +1 , limit_run ) == "org" ):
                    data.Tag[ run ] = "org_cont"
                else:
                    data.Tag[ run ] = "org_end"
            elif( helper_get_tag( data , run , +1 , limit_run ) == "org" ):
                data.Tag[ run ] = "org_start"
        elif data.Tag[ run ] == "per" :
            if( helper_get_tag( data , run , -1 , limit_run ) in ( "per_start" , "per_cont" ) ):
                if( helper_get_tag( data , run , +1, limit_run ) == "per" ):
                    data.Tag[ run ] = "per_cont"
                else:
                    data.Tag[ run ] = "per_end"
            elif( helper_get_tag( data , run , +1 , limit_run ) == "per" ):
                data.Tag[ run ] = "per_start"
            else:
                None
        else:
            None

def convert_list_to_data_frame( data_frame , data_list ):
    save_sentence = 0
    major_index_list = 0
    minor_index_list = 0
    for run in range( 0 , data_frame.shape[0] ):
        temp_sentence = num_of_sentence( data_frame["Sentence #"][ run ] )
        if save_sentence != temp_sentence :
            major_index_list = temp_sentence - 1
            save_sentence = temp_sentence
            minor_index_list = 0
        data_frame["Tag"][run] = data_list[ major_index_list][ minor_index_list ]
        minor_index_list += 1

In [6]:
def prepare_dictionary( directory , file_dict ):
    dictionary = {}
    for key in file_dict.keys():
        dictionary[ key ] = []
        list_file = file_dict[ key ]
        if type( list_file ) == type( "test_string" ):
            dictionary[ key ] = read_all_line( directory , list_file ).split('\n')
        else:
            for file in list_file:
                dictionary[ key ] += read_all_line( directory , file ).split('\n')
    return dictionary

# If found in dict will return key other return NO
def search_in_dictionary( word , dictionary ):
    answer = "NO"
    for key in dictionary.keys():
        if word in dictionary[key]:
            answer = key
            break
    return answer

In [7]:
dictionary_directory = "dictionary_directory"
dictionary_files = {
    "front_person" : ("clue front.txt" , "clue word person.txt" , "clue_royal.txt")
    ,"front_country" : ("คำนำหน้าชื่อประเทศ.txt")
    ,"front_org" : ("คำนำหน้าองค์กรจาก dict.txt")
    ,"location_name" : ("ชื่อกิ่งอำเภอ.txt" , "ชื่อคลอง.txt" , "ชื่อจังหวัด.txt" , "ชื่อตำบล.txt" , "ชื่อมลรัฐ.txt" , "ชื่อสถานที่.txt")
}
dictionary = prepare_dictionary( dictionary_directory , dictionary_files )

In [8]:
def idividual_read_file( raw_data , split_sentence , count ,remove = [] ):
    pre_data_frame = { "Sentence #" : [] , "Word" : [] , "Tag" : [] , "Dictionary" : [] }
    for word in raw_data:
        if word in remove :
            None
        elif word in split_sentence :
            count += 1
        else: 
            check = word.find('(')
            if check > 0:
                temp_word = word[ 0 : word.find('(')]
                pre_data_frame["Sentence #"].append( "Sentence: " + str(count) )
                pre_data_frame["Word"].append( temp_word )
                pre_data_frame["Tag"].append( word[ word.find('(') + 1 : word.find(')')] )
                pre_data_frame["Dictionary"].append( search_in_dictionary( temp_word , dictionary ) )
            else:
                pre_data_frame["Sentence #"].append( "Sentence: " + str(count) )
                pre_data_frame["Word"].append( word )
                pre_data_frame["Tag"].append( "O" )
                pre_data_frame["Dictionary"].append( search_in_dictionary( word , dictionary ) )
    return pre_data_frame , count

def read_file( directory , list_file , count = 1 , individual = False ):
    data_frame = pd.DataFrame( { "Sentence #" : [] , "Word" : [] , "Tag" : [] , "Dictionary" : []} )
    if( individual ):
        word = read_all_line( directory , list_file ).split('|')
        pre_data_frame , count = idividual_read_file( word , # raw_data
                                              ["\n"] , # word show split sentence
                                              count , # order of sentence
                                              [ " " , '' , '\0'] ) # word to delete or prevent
        data_frame  = data_frame.append( pd.DataFrame( pre_data_frame ), ignore_index=True )
    else:
        for file in list_file :
            word = read_all_line( directory , file ).split('|')
            pre_data_frame , count = idividual_read_file( word ,
                                              ["\n"] ,
                                              count ,
                                              [ " " , '' , '\0'] )
            data_frame  = data_frame.append( pd.DataFrame( pre_data_frame ), ignore_index=True )
    return data_frame
def num_of_sentence( sentence ):
    words = sentence.split( " ")
    return int( words[1] )

In [9]:
# Variable for collect name file and access file
train_directory = "corpus_directory"
train_files = ("Politic9.txt" , "Allcolumn.txt" , "98JUL5_1.txt" ,
                "98JUL5_2.txt" , "bkknews1.txt" , "Result1.txt" ,
                "Result2(corpus-1).txt" , "Result3(corpus-2).txt" )
test_directory = "corpus_directory"
test_files = ("result4.txt" , "Result_c_2_1.txt", "Result_c_2_2.txt" )

In [10]:
data = read_file( train_directory , train_files , 1 )
data.fillna( method="ffill")

,Sentence #,Word,Tag,Dictionary
0,Sentence: 1,"""",O,NO
1,Sentence: 1,พลอากาศเอก,O,front_person
2,Sentence: 1,ทักษิณ,per,NO
3,Sentence: 1,"""",O,NO
4,Sentence: 1,ขอ,O,NO
...,...,...,...,...
52217,Sentence: 3762,ประกาศ,O,NO
52218,Sentence: 3762,หลักเกณฑ์,O,NO
52219,Sentence: 3762,และ,O,location_name
52220,Sentence: 3762,จำนวน,O,NO


In [11]:
old_tag_data = merge_all_tag( data )

In [12]:
getter = SentenceGetter( data )
sentences = getter.sentences

In [13]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [14]:
crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)
crf.fit(X, y)

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [15]:
model_directory = "model_directory"
model_name = "CRF_dictionary.sav"
model_file = model_directory + "/" + model_name

In [16]:
pickle.dump( crf , open( model_file , 'wb') )

In [17]:
data_test = read_file( test_directory , test_files , 1 )
data_test.fillna( method="ffill");

In [18]:
old_data_tag_test = merge_all_tag( data_test )

In [19]:
getter_test = SentenceGetter( data_test )
sentences_test = getter_test.sentences

In [20]:
test_set = [sent2features(s) for s in sentences_test]
test_predict = crf.predict( test_set )
convert_list_to_data_frame( data_test , test_predict )
split_all_tag( data_test )

In [21]:
data_test

,Sentence #,Word,Tag,Dictionary
0,Sentence: 1,มา,O,NO
1,Sentence: 1,พูด,O,NO
2,Sentence: 1,คุย,O,NO
3,Sentence: 1,กัน,O,NO
4,Sentence: 1,ถึง,O,NO
...,...,...,...,...
36377,Sentence: 2064,และ,org_cont,location_name
36378,Sentence: 2064,ประมง,org_cont,NO
36379,Sentence: 2064,ทะเล,org_end,NO
36380,Sentence: 2064,จังหวัด,O,NO


In [22]:
test_predict = get_tag_group_sentence( data_test )

In [23]:
test_predict

[['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['org_start'],
 ['org_cont'],
 ['org_end'],
 ['O'],
 ['org'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['org'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['org'],
 ['O'],
 ['O'],
 ['O'],
 ['per'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['org'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['org_start'],
 ['org_cont'],
 ['org_cont'],
 ['org_end'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['O'],
 ['per_start'],
 ['per_cont']

In [24]:
report = flat_classification_report(y_pred=test_predict, y_true=old_data_tag_test)
print(report)

              precision    recall  f1-score   support

           O       0.96      0.99      0.97     29673
         loc       0.90      0.58      0.71      1127
    loc_cont       0.11      0.19      0.14        54
     loc_end       0.35      0.46      0.40        74
   loc_start       0.39      0.51      0.44        74
         org       0.84      0.61      0.71      1176
    org_cont       0.79      0.71      0.75      1027
     org_end       0.71      0.65      0.68       656
   org_start       0.73      0.66      0.69       671
         per       0.88      0.41      0.56       331
    per_cont       0.83      0.89      0.86       541
     per_end       0.91      0.85      0.88       489
   per_start       0.91      0.85      0.88       489

    accuracy                           0.93     36382
   macro avg       0.72      0.64      0.67     36382
weighted avg       0.93      0.93      0.93     36382

